# <center>**Notebook get_events()**</center>

Notebook em jupyter que descreve a função `get_events()` do scrip ufc_scap() desde o webscraping até os tratamentos realizados para gerar o data frame final.

### Bibliotecas utilizadas

In [1]:
# importando bibliotecas utilizadas no projeto
# Webscraping
import requests 
from bs4 import BeautifulSoup

# Manipulação de dados
import pandas as pd
import numpy as np

## Webscraping

O webcraping realizado nesta função retorna o html da tabela para transformação em dataframe.

In [2]:
# Iniciando função
def get_events():
    """
        Função que retorna a tabela de eventos realizados no ufc em formato html
    """
    # Link de acesso para pagina
    URL = "http://www.ufcstats.com/statistics/events/completed?page=all"


    # Realizando o request para o site 

    response = requests.get(URL)
    response.raise_for_status()

    # Obtendo pagina completa em html
    soup = BeautifulSoup(response.content, "html.parser")

    # Encontrando a tabela com a classe especificada

    table = soup.find("table", class_="b-statistics__table-events")

    return(table)

Nesta parte do código estamos obtendo a tabela em formato html.

In [3]:
# Mostrando resultado
tabela = get_events()

type(tabela)

bs4.element.Tag

In [4]:
# Visualizando resultado
tabela

<table class="b-statistics__table-events">
<thead class="b-statistics__table-caption">
<tr class="b-statistics__table-row">
<th class="b-statistics__table-col">
                  Name/date
                </th>
<th class="b-statistics__table-col">
                  Location
                </th>
</tr>
</thead>
<tbody>
<tr class="b-statistics__table-row">
<td class="b-statistics__table-col_type_clear"></td>
</tr>
<tr class="b-statistics__table-row_type_first">
<td class="b-statistics__table-col">
<img class="b-statistics__icon" src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/next.png">
<i class="b-statistics__table-content">
<a class="b-link b-link_style_white" href="http://www.ufcstats.com/event-details/ece280745f8727b8">
                          UFC 293: Adesanya vs. Strickland
                        </a>
<span class="b-statistics__date">
                          September 09, 2023
                        </span>
</i>
</img></td>
<td class="b-s

## Tratamento

Para poder utilizar estes dados na nossa analise e modelagem, primeiro é necessario transformar esta tabela html em um data frame pandas.

In [5]:
# Alterando função para transformar tabela em dataframe
def get_events():
    """
        Função que retorna a tabela de eventos realizados no ufc em formato html
    """
    # Link de acesso para pagina
    URL = "http://www.ufcstats.com/statistics/events/completed?page=all"


    # Realizando o request para o site
    response = requests.get(URL)
    response.raise_for_status()

    # Obtendo pagina completa em html
    soup = BeautifulSoup(response.content, "html.parser")

    # Encontrando a tabela com a classe especificada
    table = soup.find("table", class_="b-statistics__table-events")

    # Transformando tabela de html para dataframe pandas ou retornando erro se não encontrar a tabela
    if table:
        # Extraindo cabeçalho
        header = [th.text.strip() for th in table.find('thead').find_all('th')]

        # Extraindo as linhas
        rows = table.find('tbody').find_all('tr')
        table_data = [[col.text.strip() for col in row.find_all('td')] for row in rows]

        # Convertendo em dataframe
        df_bruto = pd.DataFrame(table_data[1:], columns=header) #Obs: Não considera a primeira linha

        return df_bruto
    else:
        return print("Não foi possível encontrar a tabela desejada na página.")

Com as alterações feitas na função get_events(), ela agora retorna o dataframe em sua forma original, sem qualquer processamento. Se a tabela não for localizada, uma mensagem de erro é exibida.

`Observação:`
    É importante mencionar que há um leve pré-processamento ao criar o dataframe, onde a primeira linha é ignorada. Isso ocorre porque, no formato atual da tabela no dia em que este script foi feito, essa linha é consistentemente nula.

In [6]:
# Visualizando o retorno
df = get_events()

df

,Name/date,Location
0,UFC 293: Adesanya vs. Strickland\n ...,"Sydney, New South Wales, Australia"
1,UFC Fight Night: Gane vs. Spivac\n ...,"Paris, Ile-de-France, France"
2,UFC Fight Night: Holloway vs. The Korean Zombi...,"Kallang, Singapore"
3,UFC 292: Sterling vs. O'Malley\n ...,"Boston, Massachusetts, USA"
4,UFC Fight Night: Luque vs. Dos Anjos\n ...,"Las Vegas, Nevada, USA"
...,...,...
657,UFC 6: Clash of the Titans\n ...,"Casper, Wyoming, USA"
658,UFC 5: The Return of the Beast\n ...,"Charlotte, North Carolina, USA"
659,UFC 4: Revenge of the Warriors\n ...,"Tulsa, Oklahoma, USA"
660,UFC 3: The American Dream\n ...,"Charlotte, North Carolina, USA"


Ao observar o dataframe bruto, notamos que ele tem duas colunas. Mas, ao examinar o conteúdo da primeira coluna, identificamos duas informações chave: o nome do evento e a data. Ao tratar essas informações, poderemos reestruturar o dataframe de modo que apresente colunas distintas para o nome do evento e sua respectiva data.

Para entender melhor o processo de tratamento, será analisada a string:

In [7]:
# Criando variavel e armazenando a primeira linha
string_name_date = df['Name/date'][0]

string_name_date = string_name_date.strip().split("\n")[-1].strip()

string_name_date

'September 09, 2023'

Realizando este tratamento, temos a data.

In [8]:
# Transformando em datetime
datetime_name_date = pd.to_datetime(string_name_date).date()

display(datetime_name_date)
type(datetime_name_date)

datetime.date(2023, 9, 9)

datetime.date

Com estes tratamentos, temos duas novas colunas com a data e nome do evento.

In [9]:
# Aplicando no dataframe para visualizar o resultado

# Ajustando o evento
df['event'] = df['Name/date'].str.split("\n")

df['event'] = df['event'].apply(lambda x: x[0])

# Ajustando data
df['date'] = df['Name/date'].apply(lambda x: x.split("\n")[-1].strip())

df['date'] = pd.to_datetime(df['date'])

df

,Name/date,Location,event,date
0,UFC 293: Adesanya vs. Strickland\n ...,"Sydney, New South Wales, Australia",UFC 293: Adesanya vs. Strickland,2023-09-09
1,UFC Fight Night: Gane vs. Spivac\n ...,"Paris, Ile-de-France, France",UFC Fight Night: Gane vs. Spivac,2023-09-02
2,UFC Fight Night: Holloway vs. The Korean Zombi...,"Kallang, Singapore",UFC Fight Night: Holloway vs. The Korean Zombie,2023-08-26
3,UFC 292: Sterling vs. O'Malley\n ...,"Boston, Massachusetts, USA",UFC 292: Sterling vs. O'Malley,2023-08-19
4,UFC Fight Night: Luque vs. Dos Anjos\n ...,"Las Vegas, Nevada, USA",UFC Fight Night: Luque vs. Dos Anjos,2023-08-12
...,...,...,...,...
657,UFC 6: Clash of the Titans\n ...,"Casper, Wyoming, USA",UFC 6: Clash of the Titans,1995-07-14
658,UFC 5: The Return of the Beast\n ...,"Charlotte, North Carolina, USA",UFC 5: The Return of the Beast,1995-04-07
659,UFC 4: Revenge of the Warriors\n ...,"Tulsa, Oklahoma, USA",UFC 4: Revenge of the Warriors,1994-12-16
660,UFC 3: The American Dream\n ...,"Charlotte, North Carolina, USA",UFC 3: The American Dream,1994-09-09


Após estas modificações, podemos retirar a coluna Name/date

In [10]:
# Retirando coluna Name/date

df = df.drop(columns=["Name/date"])

df

,Location,event,date
0,"Sydney, New South Wales, Australia",UFC 293: Adesanya vs. Strickland,2023-09-09
1,"Paris, Ile-de-France, France",UFC Fight Night: Gane vs. Spivac,2023-09-02
2,"Kallang, Singapore",UFC Fight Night: Holloway vs. The Korean Zombie,2023-08-26
3,"Boston, Massachusetts, USA",UFC 292: Sterling vs. O'Malley,2023-08-19
4,"Las Vegas, Nevada, USA",UFC Fight Night: Luque vs. Dos Anjos,2023-08-12
...,...,...,...
657,"Casper, Wyoming, USA",UFC 6: Clash of the Titans,1995-07-14
658,"Charlotte, North Carolina, USA",UFC 5: The Return of the Beast,1995-04-07
659,"Tulsa, Oklahoma, USA",UFC 4: Revenge of the Warriors,1994-12-16
660,"Charlotte, North Carolina, USA",UFC 3: The American Dream,1994-09-09


Adicionalmente ao tratamento já mencionado, faremos uma modificação na coluna "location". Separando a cidade e o pais em duas colunas distintas.

In [11]:
# Criando string para demonstração do código
string_location = df['Location'][0]

string_location = string_location.split(",")

string_location

['Sydney', ' New South Wales', ' Australia']

Ao usar a função "split", conseguimos criar colunas para a cidade, estado e país. É importante destacar que algumas entradas na coluna original não apresentam o estado. Por conta disto, foi implementada uma condição que insere valores nulos quando o estado não está especificado.

In [12]:
# Aplicando modificações no data frame
df['city'] = df['Location'].apply(lambda x: x.split(",")[0].strip())

df['country'] = df['Location'].apply(lambda x: x.split(",")[-1].strip())

df['state'] = df['Location'].apply(lambda x: x.split(",")[1].strip() if len(x.split(",")) == 3 else np.nan)

df

,Location,event,date,city,country,state
0,"Sydney, New South Wales, Australia",UFC 293: Adesanya vs. Strickland,2023-09-09,Sydney,Australia,New South Wales
1,"Paris, Ile-de-France, France",UFC Fight Night: Gane vs. Spivac,2023-09-02,Paris,France,Ile-de-France
2,"Kallang, Singapore",UFC Fight Night: Holloway vs. The Korean Zombie,2023-08-26,Kallang,Singapore,NaN
3,"Boston, Massachusetts, USA",UFC 292: Sterling vs. O'Malley,2023-08-19,Boston,USA,Massachusetts
4,"Las Vegas, Nevada, USA",UFC Fight Night: Luque vs. Dos Anjos,2023-08-12,Las Vegas,USA,Nevada
...,...,...,...,...,...,...
657,"Casper, Wyoming, USA",UFC 6: Clash of the Titans,1995-07-14,Casper,USA,Wyoming
658,"Charlotte, North Carolina, USA",UFC 5: The Return of the Beast,1995-04-07,Charlotte,USA,North Carolina
659,"Tulsa, Oklahoma, USA",UFC 4: Revenge of the Warriors,1994-12-16,Tulsa,USA,Oklahoma
660,"Charlotte, North Carolina, USA",UFC 3: The American Dream,1994-09-09,Charlotte,USA,North Carolina


Após estas modificações, podemos retirar a coluna Location e chegaremos no dataframe final.

In [13]:
# Retirando a coluna Location
df = df.drop(columns=["Location"])

df

,event,date,city,country,state
0,UFC 293: Adesanya vs. Strickland,2023-09-09,Sydney,Australia,New South Wales
1,UFC Fight Night: Gane vs. Spivac,2023-09-02,Paris,France,Ile-de-France
2,UFC Fight Night: Holloway vs. The Korean Zombie,2023-08-26,Kallang,Singapore,NaN
3,UFC 292: Sterling vs. O'Malley,2023-08-19,Boston,USA,Massachusetts
4,UFC Fight Night: Luque vs. Dos Anjos,2023-08-12,Las Vegas,USA,Nevada
...,...,...,...,...,...
657,UFC 6: Clash of the Titans,1995-07-14,Casper,USA,Wyoming
658,UFC 5: The Return of the Beast,1995-04-07,Charlotte,USA,North Carolina
659,UFC 4: Revenge of the Warriors,1994-12-16,Tulsa,USA,Oklahoma
660,UFC 3: The American Dream,1994-09-09,Charlotte,USA,North Carolina


Após a etapa de webscraping e os devidos ajustes, este é o formato do dataframe.

Considerando a intenção é tambem poder usar esta tabela em um banco de dados relacional, adicionaremos uma coluna para as chaves únicas de cada evento.


Estrutura das chaves:

<center>EVEXXXX</center>


Onde "EVE" indica a origem da tabela de eventos e "XXXX" é o número sequencial do evento. Por exemplo, o código EVE0001 denota o primeiro evento listado na tabela do site em questão.

In [14]:
# Criando range com o index invertido

inverted_index = df.index[::-1] + 1

# Criando a chave primaria no formato explicado

df['event_id'] = ["EVE" + f"{index:04}" for index in inverted_index]

df


,event,date,city,country,state,event_id
0,UFC 293: Adesanya vs. Strickland,2023-09-09,Sydney,Australia,New South Wales,EVE0662
1,UFC Fight Night: Gane vs. Spivac,2023-09-02,Paris,France,Ile-de-France,EVE0661
2,UFC Fight Night: Holloway vs. The Korean Zombie,2023-08-26,Kallang,Singapore,NaN,EVE0660
3,UFC 292: Sterling vs. O'Malley,2023-08-19,Boston,USA,Massachusetts,EVE0659
4,UFC Fight Night: Luque vs. Dos Anjos,2023-08-12,Las Vegas,USA,Nevada,EVE0658
...,...,...,...,...,...,...
657,UFC 6: Clash of the Titans,1995-07-14,Casper,USA,Wyoming,EVE0005
658,UFC 5: The Return of the Beast,1995-04-07,Charlotte,USA,North Carolina,EVE0004
659,UFC 4: Revenge of the Warriors,1994-12-16,Tulsa,USA,Oklahoma,EVE0003
660,UFC 3: The American Dream,1994-09-09,Charlotte,USA,North Carolina,EVE0002


O ultimo tratamento é em reajustar a ordem das colunas.

In [15]:
# Reordenando colunas do dataframe

df = df[["event_id", "event", "date","city", "state", "country"]]

df

,event_id,event,date,city,state,country
0,EVE0662,UFC 293: Adesanya vs. Strickland,2023-09-09,Sydney,New South Wales,Australia
1,EVE0661,UFC Fight Night: Gane vs. Spivac,2023-09-02,Paris,Ile-de-France,France
2,EVE0660,UFC Fight Night: Holloway vs. The Korean Zombie,2023-08-26,Kallang,NaN,Singapore
3,EVE0659,UFC 292: Sterling vs. O'Malley,2023-08-19,Boston,Massachusetts,USA
4,EVE0658,UFC Fight Night: Luque vs. Dos Anjos,2023-08-12,Las Vegas,Nevada,USA
...,...,...,...,...,...,...
657,EVE0005,UFC 6: Clash of the Titans,1995-07-14,Casper,Wyoming,USA
658,EVE0004,UFC 5: The Return of the Beast,1995-04-07,Charlotte,North Carolina,USA
659,EVE0003,UFC 4: Revenge of the Warriors,1994-12-16,Tulsa,Oklahoma,USA
660,EVE0002,UFC 3: The American Dream,1994-09-09,Charlotte,North Carolina,USA


Reajustando função final:

In [19]:
def get_events(bruto=False):
    """
    Função que retorna o data frame com todos os eventos realizados pelo UFC e armazenados no site UFCstats.
    """
    # Link de acesso para pagina
    URL = "http://www.ufcstats.com/statistics/events/completed?page=all"


    # Realizando o request para o site 
    response = requests.get(URL)
    response.raise_for_status()

    # Obtendo pagina completa em html
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Encontrando a tabela com a classe especificada
    table = soup.find("table", class_="b-statistics__table-events")

    # Transformando tabela de html para dataframe pandas ou retornando erro se não encontrar a tabela
    if table:
        # Extraindo cabeçalho
        header = [th.text.strip() for th in table.find('thead').find_all('th')]

        # Extraindo as linhas
        rows = table.find('tbody').find_all('tr')
        table_data = [[col.text.strip() for col in row.find_all('td')] for row in rows]

        # Convertendo em dataframe
        df_bruto = pd.DataFrame(table_data[1:], columns=header) #Obs: Não considera a primeira linha

    else:
        return print("Não foi possível encontrar a tabela desejada na página.")


    if bruto:
        return df_bruto

    """
    A parte do código abaixo é dedicata ao tratamento do df_bruto para os parametros pré estabelecidos no arquivo note_get_events.ipynb.
    """

    # Criando o dataframe final
    df_final = df_bruto.copy()

    # Ajustando coluna evento
    df_final['event'] = df_final['Name/date'].str.split("\n")
    df_final['event'] = df_final['event'].apply(lambda x: x[0])

    # Ajustando data
    df_final['date'] = df_final['Name/date'].apply(lambda x: x.split("\n")[-1].strip())
    df_final['date'] = pd.to_datetime(df_final['date'])

    # Retirando coluna Name/date
    df_final = df_final.drop(columns=["Name/date"])

    # Criando colunas de cidade, estado e pais
    # Cidade
    df_final['city'] = df_final['Location'].apply(lambda x: x.split(",")[0].strip())

    # Estado
    df_final['state'] = df_final['Location'].apply(lambda x: x.split(",")[1].strip() if len(x.split(",")) == 3 else np.nan)

    # Pais
    df_final['country'] = df_final['Location'].apply(lambda x: x.split(",")[-1].strip())

    # Removendo a coluna Location
    df_final = df_final.drop(columns=["Location"])


    """
    A parte do código a seguir, cria uma primary key para a tabela de evetos, para mais detalhes olhar o arquivo documentado note_get_events.ipynb
    """

    # Criando range com o index invertido
    inverted_index = df_final.index[::-1] + 1

    # Criando a chave primaria no formato EVEXXXX
    df_final['event_id'] = ["EVE" + f"{index:04}" for index in inverted_index]

    """
    Como ultimo tratamento, as colunas serão reorganizadas
    """
    df_final = df_final[["event_id", "event", "date","city", "state", "country"]]

    return df_final



## Dicas de uso

- Obtendo dataframe:

In [20]:
# Criando dataframe de eventos utilizando a função
df_events = get_events()

df_events

,event_id,event,date,city,state,country
0,EVE0663,UFC Fight Night: Grasso vs. Shevchenko 2,2023-09-16,Las Vegas,Nevada,USA
1,EVE0662,UFC 293: Adesanya vs. Strickland,2023-09-09,Sydney,New South Wales,Australia
2,EVE0661,UFC Fight Night: Gane vs. Spivac,2023-09-02,Paris,Ile-de-France,France
3,EVE0660,UFC Fight Night: Holloway vs. The Korean Zombie,2023-08-26,Kallang,NaN,Singapore
4,EVE0659,UFC 292: Sterling vs. O'Malley,2023-08-19,Boston,Massachusetts,USA
...,...,...,...,...,...,...
658,EVE0005,UFC 6: Clash of the Titans,1995-07-14,Casper,Wyoming,USA
659,EVE0004,UFC 5: The Return of the Beast,1995-04-07,Charlotte,North Carolina,USA
660,EVE0003,UFC 4: Revenge of the Warriors,1994-12-16,Tulsa,Oklahoma,USA
661,EVE0002,UFC 3: The American Dream,1994-09-09,Charlotte,North Carolina,USA


- Obtendo Dataframe Bruto

In [22]:
# Obtendo dataframe bruto, sem processamento
df_bruto = get_events(bruto=True)
df_bruto

,Name/date,Location
0,UFC Fight Night: Grasso vs. Shevchenko 2\n ...,"Las Vegas, Nevada, USA"
1,UFC 293: Adesanya vs. Strickland\n ...,"Sydney, New South Wales, Australia"
2,UFC Fight Night: Gane vs. Spivac\n ...,"Paris, Ile-de-France, France"
3,UFC Fight Night: Holloway vs. The Korean Zombi...,"Kallang, Singapore"
4,UFC 292: Sterling vs. O'Malley\n ...,"Boston, Massachusetts, USA"
...,...,...
658,UFC 6: Clash of the Titans\n ...,"Casper, Wyoming, USA"
659,UFC 5: The Return of the Beast\n ...,"Charlotte, North Carolina, USA"
660,UFC 4: Revenge of the Warriors\n ...,"Tulsa, Oklahoma, USA"
661,UFC 3: The American Dream\n ...,"Charlotte, North Carolina, USA"


## Atualizações

Sem atualizações até o momento.